In [13]:
from ipywidgets import widgets
from IPython.display import clear_output
from IPython.display import display
from IPython.display import HTML
import json
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import jieba
import jieba.analyse
import logging
jieba_tracer = logging.getLogger('jieba')
jieba_tracer.disabled = True
jieba.set_dictionary('./config/google_dic.dic')
jieba.analyse.set_stop_words('./config/merged_stopword.txt')
from collections import Counter
from collections import OrderedDict


def word_seg(content, top_K):
    seg_list = list(jieba.cut(content, cut_all=False))
    c = Counter(seg_list)
    new_c = {k: v for k, v in c.items() if len(k) > 1 and v is not 1}
    new_sc = OrderedDict(sorted(dict(new_c).items(), key=lambda t: t[1], reverse=True))
    if top_K > len(new_sc):
        top_K = len(new_sc)
    words = [k for k in new_sc.keys()[:top_K]]
    return words

with open('news_data.json', 'rb') as fin:
    result = json.load(fin)

news_list = []
news_dict = {}
for row in result:
    d = {}
    d[u'id'] = row['id']
    d[u'來源'] = row['from_name']
    d[u'標題'] = row['title']
    d[u'內容'] = row['content']
    d[u'網址'] = row['url']
    d[u'類型'] = row['type']
    news_list.append(d)
    news_dict[row['id']] = d

    
def update_chk_list(x):
    for tag in c_tags:
        news_id = tags_map[tag.model_id]['id']
        custom_tags[news_id] = [x for x in tag.value.split(',')]
            
        for chk in chk_map[news_id]:
            if chk.value == False:
                chk_map[news_id].remove(chk)

    clear_output()
    show_form(update=True)



c_tags = []
tags_map = {}
custom_tags = {}
chk_map = {}
def show_form(update=False):
    n = 0
    update_btn = widgets.Button(description='Update')
    update_btn.on_click(update_chk_list)
    display(update_btn)
    for news in news_list[:5]:
        df_news = pd.DataFrame(list(news.values()))
        container = widgets.Box()

        d = df_news.to_html(escape=False, header=False, index = False)

        br = widgets.HTML("---------------------------------------------")
        news_info = '{0} <a href="{1}" target="_blank">link</a>'.format(news[u'標題'].encode('utf-8'), news[u'網址'])
        link = widgets.HTML(news_info.decode('utf-8'))
        news_id = news[u'id']
        news_title = news[u'標題']

        

        form = widgets.VBox()
        custom_tag = widgets.Text(description=u"自訂tag:", value="")

        words = word_seg(news[u'內容'], 10)
        chk_list = []
        if update == False:
            for word in words:
                chk = widgets.Checkbox(description=word, value=True)
                chk_list.append(chk)
        else:
            for chk_tag in chk_map[news_id]:
                chk = widgets.Checkbox(description=chk_tag.description, value=True)
                chk_list.append(chk)
            
        c_tags.append(custom_tag)
        
        if len(custom_tags) > 0 and len(custom_tags[news_id]) > 0:
            print custom_tags[news_id]
            for tag in custom_tags[news_id]:
                if len(tag) > 0:
                    chk = widgets.Checkbox(description=tag, value=True)
                    chk_list.append(chk)
            
        n += 1
        
        tags_map[custom_tag.model_id] = {'id': news_id, 'title':news_title}

        container = widgets.VBox(children=[br,link,custom_tag])
        container_chk = widgets.VBox(children=chk_list)
        chk_map[news_id] = chk_list
        display(container)
        display(container_chk)
        
show_form()

Button(description=u'Update', style=ButtonStyle())

VBox(children=(HTML(value=u'---------------------------------------------'), HTML(value=u'\u5750\u5728\u96f2\u7aef\u4e0a\u5403\u5b89\u683c\u65af\u725b\uff01\u53f0\u5317\u6700\u9ad8\u7684\u300c\u5403\u5230\u98fd\u300d\u9910\u5ef3\u958b\u5e55 <a href="http://travel.ettoday.net/article/739230.htm" target="_blank">link</a>'), Text(value=u'', description=u'\u81ea\u8a02tag:')))

VBox(children=(Checkbox(value=True, description=u'\u53f0\u5317'), Checkbox(value=True, description=u'\u63d0\u4f9b'), Checkbox(value=True, description=u'\u677e\u5c71'), Checkbox(value=True, description=u'\u610f\u820d'), Checkbox(value=True, description=u'\u9152\u5e97'), Checkbox(value=True, description=u'\u9910\u5ef3'), Checkbox(value=True, description=u'\u9078\u64c7'), Checkbox(value=True, description=u'Que'), Checkbox(value=True, description=u'\u81ea\u52a9\u9910'), Checkbox(value=True, description=u'\u76ce\u53f8')))

VBox(children=(HTML(value=u'---------------------------------------------'), HTML(value=u'\u611b\u5b78\u4eba...\u4e2d\u570b\u4eba\u5fae\u535a\u767c\u8d77\u300c\u5411\u53f0\u7063\u9053\u6b49\u300d\u5927\u8cfd\u56de\u64ca <a href="http://news.ltn.com.tw/news/world/breakingnews/1768620" target="_blank">link</a>'), Text(value=u'', description=u'\u81ea\u8a02tag:')))

VBox(children=(Checkbox(value=True, description=u'\u4e2d\u570b'), Checkbox(value=True, description=u'\u53f0\u7063'), Checkbox(value=True, description=u'\u6211\u5011'), Checkbox(value=True, description=u'\u7db2\u53cb'), Checkbox(value=True, description=u'\u5fae\u535a'), Checkbox(value=True, description=u'\u767c\u8d77'), Checkbox(value=True, description=u'\u7b2c\u4e00\u5c46'), Checkbox(value=True, description=u'\u6b49\u5927\u8cfd'), Checkbox(value=True, description=u'\u4e0d\u8d77'), Checkbox(value=True, description=u'GDP')))

...

VBox(children=(Checkbox(value=True, description=u'\u89d2\u8272'), Checkbox(value=True, description=u'\u5e73\u7b49'), Checkbox(value=True, description=u'\u7238\u7238'), Checkbox(value=True, description=u'\u5e02\u5e9c'), Checkbox(value=True, description=u'\u63d0\u524d'), Checkbox(value=True, description=u'\u6731\u7acb\u502b'), Checkbox(value=True, description=u'\u98b1\u98a8'), Checkbox(value=True, description=u'\u9700\u8981'), Checkbox(value=True, description=u'\u5fc5\u9808'), Checkbox(value=True, description=u'\u5fb7\u570b')))

VBox(children=(HTML(value=u'---------------------------------------------'), HTML(value=u'\u985b\u8986\u8fb2\u696d\u523b\u677f\u5370\u8c61\uff01\u4ed6\u9760\u81ea\u5b78\u7a2e\u51fa\u852c\u679c\u50f9\u503c <a href="http://www.gvm.com.tw/webonly_content_10217.html" target="_blank">link</a>'), Text(value=u'', description=u'\u81ea\u8a02tag:')))

VBox(children=(Checkbox(value=True, description=u'\u9060\u898b'), Checkbox(value=True, description=u'\u96dc\u8a8c'), Checkbox(value=True, description=u'...'), Checkbox(value=True, description=u'\u6587\u7ae0'), Checkbox(value=True, description=u'\u6703\u54e1'), Checkbox(value=True, description=u'\u8a02\u95b1'), Checkbox(value=True, description=u'\u670d\u52d9'), Checkbox(value=True, description=u'\u5065\u5eb7'), Checkbox(value=True, description=u'\u7522\u696d'), Checkbox(value=True, description=u'\u53f0\u7063')))

VBox(children=(HTML(value=u'---------------------------------------------'), HTML(value=u'\u98a8\u8a55\uff1a\u53c8\u662f\u904a\u89bd\u8eca\uff01\u53f0\u7063\u4eba\u9084\u6562\u8b1b\u54c1\u8cea\u55ce\uff1f <a href="https://tw.news.yahoo.com/%E9%A2%A8%E8%A9%95-%E5%8F%88%E6%98%AF%E9%81%8A%E8%A6%BD%E8%BB%8A-%E5%8F%B0%E7%81%A3%E4%BA%BA%E9%82%84%E6%95%A2%E8%AC%9B%E5%93%81%E8%B3%AA%E5%97%8E-000000560.html" target="_blank">link</a>'), Text(value=u'', description=u'\u81ea\u8a02tag:')))

VBox(children=(Checkbox(value=True, description=u'\u904a\u89bd\u8eca'), Checkbox(value=True, description=u'\u696d\u8005'), Checkbox(value=True, description=u'\u53f0\u7063'), Checkbox(value=True, description=u'\u9678\u5ba2'), Checkbox(value=True, description=u'\u5169\u5cb8'), Checkbox(value=True, description=u'\u5b89\u5168'), Checkbox(value=True, description=u'\u65c5\u904a'), Checkbox(value=True, description=u'\u4ea4\u6d41'), Checkbox(value=True, description=u'\u89c0\u5149'), Checkbox(value=True, description=u'\u53f0\u7063\u4eba')))

In [12]:
for news in news_list[:5]:
    print news[u'標題']
    news_id = news[u'id']
    for chk in chk_map[news_id]:
        print chk.description, chk.value 
    print '----'


坐在雲端上吃安格斯牛！台北最高的「吃到飽」餐廳開幕
台北 True
提供 True
酒店 True
餐廳 True
選擇 True
自助餐 True
盎司 True
----
愛學人...中國人微博發起「向台灣道歉」大賽回擊
我們 True
網友 True
微博 True
發起 True
第一屆 True
歉大賽 True
不起 True
GDP True
----
朱立倫抱怨 颱風讓他無法扮演好爸爸
角色 True
平等 True
爸爸 True
市府 True
提前 True
朱立倫 True
颱風 True
需要 True
必須 True
德國 True
----
顛覆農業刻板印象！他靠自學種出蔬果價值
遠見 True
雜誌 True
... True
文章 True
會員 True
訂閱 True
服務 True
健康 True
產業 True
台灣 True
----
風評：又是遊覽車！台灣人還敢講品質嗎？
遊覽車 True
業者 True
台灣 True
陸客 True
兩岸 True
觀光 True
台灣人 True
----
